In [ ]:
#@title
# Upload:
# module14_a_20220819.py
# electrical_grid_stability_simulated_data.csv

# 14.2. Create Automatic Python and Job Script Files for Different Nodes

In [ ]:
!wget https://raw.githubusercontent.com/mhrafiei/data/main/electrical_grid_stability_simulated_data.csv

--2022-08-19 15:19:08--  https://raw.githubusercontent.com/mhrafiei/data/main/electrical_grid_stability_simulated_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2417871 (2.3M) [text/plain]
Saving to: ‘electrical_grid_stability_simulated_data.csv’

electrical_grid_sta 100%[===================>]   2.31M  --.-KB/s    in 0.06s   

2022-08-19 15:19:08 (36.4 MB/s) - ‘electrical_grid_stability_simulated_data.csv’ saved [2417871/2417871]



In [ ]:
#@title 14.2.1. Import necessary libraries
import glob
import os
import fileinput
from shutil import copyfile



In [ ]:
#@title 14.2.2. Inputs
pool_size         = 12
num_nodes         = 25

num_comb          = 2**12 - 1
model_type        = ['SVC', 'KNC', 'DTC', 'RFC']
RTT               = [0.1,0.2, 0.3, 0.4]
RRS               = 25

filename_original = 'module14_a_20220819.py'

wall_time  = '04:59:00'
#memory_val = '5G'
partition  = 'shared'
email      = 'mrafiei1@jhu.edu'

In [ ]:
#@title 14.2.3. Function to divide combinations amongst different nodes
def fun_nodebatch(num_comb, num_nodes):
  batch_size_most = int(num_comb/num_nodes)

  counter = 0
  comb_start = []
  comb_last  = []
  for i0 in range(num_nodes):
    comb_start.append(counter)
    if i0 !=num_nodes-1:
      comb_last.append(counter + batch_size_most)
    else:
      comb_last.append(num_comb)

    counter = counter +  batch_size_most

  return comb_start, comb_last

In [ ]:
#@title 14.2.4. Python files
comb_start, comb_last = fun_nodebatch(num_comb, num_nodes)

for i0 in range(len(comb_start)):
  comb_index_start = comb_start[i0]
  comb_index_end   = comb_last[i0]

  filename_new     = 'master_{:07d}_{:07d}.py'.format(comb_index_start, comb_index_end)
  copyfile(filename_original, filename_new)

  text_search01 = 'pool_size  = 2'
  text_search02 = 'comb_index_start = 110'
  text_search03 = 'comb_index_end   = 115'
  text_search04 = "model_type = ['SVC', 'KNC', 'DTC', 'RFC']"
  text_search05 = 'RTT        = [0.1,0.2,0.3,0.4,0.5]'
  text_search06 = 'RRS        = 100'

  text_replace01 = 'pool_size  = {}'.format(pool_size)
  text_replace02 = 'comb_index_start = {}'.format(comb_index_start)
  text_replace03 = 'comb_index_end   = {}'.format(comb_index_end)
  text_replace04 = "model_type = {}".format(model_type)
  text_replace05 = 'RTT        = {}'.format(RTT)
  text_replace06 = 'RRS        = {}'.format(RRS)

  text_search    = [text_search01,  text_search02,  text_search03,  text_search04,  text_search05,  text_search06]
  text_replace   = [text_replace01, text_replace02, text_replace03, text_replace04, text_replace05, text_replace06]

  for i1 in range(len(text_search)):
    file = fileinput.FileInput(filename_new, inplace=True)
    for line in file:
      print(line.replace(text_search[i1], text_replace[i1]), end='')


In [ ]:
#@title 14.2.5. Job script files
# create job lines
line_list   = []

line_list.append("#!/bin/bash")
line_list.append("#SBATCH --account=mrafiei1")
line_list.append("#SBATCH --job-name=my_job_name")
line_list.append("#SBATCH --time=00:00:00")
line_list.append("#SBATCH --partition=partition_case")
#line_list.append("#SBATCH --nodes=1")
line_list.append("#SBATCH --cpus-per-task=XC_val")
#line_list.append("#SBATCH --mem-per-cpu=XG_val")
line_list.append("#SBATCH --mail-type=end")
line_list.append("#SBATCH --mail-user=abc@jhu.edu")
line_list.append("#SBATCH --requeue")
line_list.append("# time mpiexec ./code-mvapich.x > OUT-24log")

# line_list.append("module load python/3.7.7")
# line_list.append("python -m venv ./venv_$SLURM_JOBID")
# line_list.append("source ./venv_$SLURM_JOBID/bin/activate")
# line_list.append("pip install pandas")
# line_list.append("pip install sklearn")
# line_list.append("pip install tensorflow==2.2.0rc0")
# line_list.append("pip install matplotlib")

line_list.append("ml anaconda")
line_list.append("cd /scratch4/mrafiei1/su2022/mrafiei1/")
line_list.append("conda deactivate")
line_list.append("conda activate tf_20220422")
line_list.append("python my_code.py")
#line_list.append("rm -r ./venv_$SLURM_JOBID")

# create a temporary job file as filename_original
filename_original = "file_original.sh"
if len(glob.glob(filename_original)) == 1:
  os.remove(filename_original)

file_original     = open(filename_original, "x")

for i0 in range(len(line_list)):
  if i0 == 0:
    file_original.write("{}".format(line_list[i0]))
  else:
    file_original.write("\n{}".format(line_list[i0]))

file_original.close()

master_filenames = glob.glob('master_*')
master_filenames.sort()

if len(glob.glob("job_all.sh")) == 1:
  os.remove("job_all.sh")

file_all_jobs = open("job_all.sh","x")

for i0 in range(num_nodes):
  job_name     = "job" + master_filenames[i0][6:-3]
  filename_new =  job_name + ".sh"

  file_all_jobs.write("\n{}".format("sbatch " + filename_new))


  copyfile(filename_original, filename_new)

  text_search = []
  text_search.append('my_job_name')
  text_search.append('00:00:00')
  text_search.append('abc@jhu.edu')
  text_search.append('partition_case')
  text_search.append('XC_val')
  #text_search.append('XG_val')
  text_search.append('python my_code.py')

  text_replace = []
  text_replace.append("{}".format(job_name))
  text_replace.append("{}".format(wall_time))
  text_replace.append("{}".format(email))
  text_replace.append("{}".format(partition))
  text_replace.append("{}".format(pool_size))
  #text_replace.append("{}".format(memory_val))
  text_replace.append("{}".format('python -u ' +  master_filenames[i0]))

  for i1 in range(len(text_search)):
    file = fileinput.FileInput(filename_new, inplace=True)
    for line in file:
      print(line.replace(text_search[i1], text_replace[i1]), end='')

file_all_jobs.close ()

In [ ]:
#@title 14.2.6. Cleanups
!zip master_files.zip master_* electrical_grid_stability_simulated_data.csv
!zip job_files.zip job_*
!rm master_0* job_0* file_* job_all.sh

In [ ]:
#!rm job_* master_* file_* electrical_grid_stability_simulated_data.csv